# What's this notebook going to achieve?
I am going to build my first ever simple neural network for classifying handwritten digits with one of the most famous datasets in computer vision, the MNIST. This is a homework of *Chapter 4: Under the Hood: Training a Digit Classifer* of the *Deep Learning for Coders with fastai & Pytorch* book that I have been learning in the past few weeks.

## The outline:
1. The MNIST dataset: how computers knows a digits of number and how the input dataset is structured for Machine Learning problems
   * converting images from .png to tabular data
   * how to tell if all images are equaly sized and comparable?
3. The basics of solving linear models
   * Loading the dataset into tensor as a nested array
   * Loss functions
   * Weights, aka coefficients
   * Gradients Descend
   * Accuracy as metrics
   * The 7 steps to find the optimal weights for linear model of your dataset
   * Overfitting: identification and revention
4. Build a simple 3-layered Neural Network with a mix of linear and nonlinear models
   * Activation function
   * 2 label classification: 3 and 7
  
5. Questions to answer
   * What's the final 28x28 weights for classifying 3 and 7 looks like? Make it back to a pixel representation and see if it looks like an actual number or not

In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from fastai.vision.all import *

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Section 1: The MNIST Handwritten Digits Dataset

Data source: There're many versions of MNIST datasets on Kaggle and online, but I am going to use the sample data provided by the fast.ai library, which can be downloaded using the `untar_data(URL)` function. Let's first see where is the data.

In [3]:
URLs.MNIST_SAMPLE

'https://s3.amazonaws.com/fast-ai-sample/mnist_sample.tgz'

In [4]:
path = untar_data(URLs.MNIST_SAMPLE)

## 1.1 The Structure of the MNIST sample dataset

In [5]:
path.ls()

(#3) [Path('/root/.fastai/data/mnist_sample/valid'),Path('/root/.fastai/data/mnist_sample/labels.csv'),Path('/root/.fastai/data/mnist_sample/train')]

### 1.1.1 First look at labels

In [11]:
labels_df = pd.read_csv('/root/.fastai/data/mnist_sample/labels.csv')

In [13]:
labels_df.label.describe()

count    14434.000000
mean         0.505265
std          0.499990
min          0.000000
25%          0.000000
50%          1.000000
75%          1.000000
max          1.000000
Name: label, dtype: float64

In [53]:
# randomly select 30 samples from the labels
sample_index = np.random.randint(0, labels_df.shape[0], 10)
labels_df.iloc[sample_index].sort_index(ascending=True)

# summary
# 1. there're 14,434 samples, at least 12,359 are in training set, 12,454 starts to be the validation set. so, 85% is training, 15% is validation set. 
# 2. from the index perspective, the train and validation sets are completely separated from each other
# 3. label are 50% 1 and 50% 0, and the labels mixup in both train and validation set. but I don't know what the label means.

,name,label
1082,train/3/4632.png,0
1131,train/3/7483.png,0
3593,train/3/49789.png,0
5383,train/3/23888.png,0
7843,train/7/52089.png,1
7845,train/7/52235.png,1
8407,train/7/35773.png,1
9744,train/7/40386.png,1
13144,valid/3/8226.png,0
14433,valid/7/7149.png,1


### 1.1.2 Then let's look at what's inside train folder
1. Only contains 3 and 7. Let's create 2 lists named `sevens` and `threes`, each element of the list is the path that saves 1 image.
2. Most importantly: the images are saved as '.png' files, not numbers!

In [6]:
(path/'train').ls()

(#2) [Path('/root/.fastai/data/mnist_sample/train/7'),Path('/root/.fastai/data/mnist_sample/train/3')]

In [27]:
sevens = (path/'train'/'7').ls().sorted()
sevens

(#6265) [Path('/root/.fastai/data/mnist_sample/train/7/10002.png'),Path('/root/.fastai/data/mnist_sample/train/7/1001.png'),Path('/root/.fastai/data/mnist_sample/train/7/10014.png'),Path('/root/.fastai/data/mnist_sample/train/7/10019.png'),Path('/root/.fastai/data/mnist_sample/train/7/10039.png'),Path('/root/.fastai/data/mnist_sample/train/7/10046.png'),Path('/root/.fastai/data/mnist_sample/train/7/10050.png'),Path('/root/.fastai/data/mnist_sample/train/7/10063.png'),Path('/root/.fastai/data/mnist_sample/train/7/10077.png'),Path('/root/.fastai/data/mnist_sample/train/7/10086.png'),Path('/root/.fastai/data/mnist_sample/train/7/10095.png'),Path('/root/.fastai/data/mnist_sample/train/7/10096.png'),Path('/root/.fastai/data/mnist_sample/train/7/101.png'),Path('/root/.fastai/data/mnist_sample/train/7/10131.png'),Path('/root/.fastai/data/mnist_sample/train/7/1014.png'),Path('/root/.fastai/data/mnist_sample/train/7/10147.png'),Path('/root/.fastai/data/mnist_sample/train/7/10148.png'),Path('/ro

In [35]:
from PIL import Image
seven_rand = np.random.randint(0,len(sevens),10)
Image.open(sevens[seven_rand[0]])

In [36]:
Image.open(sevens[seven_rand[1]])

In [37]:
Image.open(sevens[seven_rand[2]])

In [41]:
Image.open(sevens[seven_rand[-1]])

In [44]:
threes = (path/'train/3').ls().sorted()
threes

(#6131) [Path('/root/.fastai/data/mnist_sample/train/3/10.png'),Path('/root/.fastai/data/mnist_sample/train/3/10000.png'),Path('/root/.fastai/data/mnist_sample/train/3/10011.png'),Path('/root/.fastai/data/mnist_sample/train/3/10031.png'),Path('/root/.fastai/data/mnist_sample/train/3/10034.png'),Path('/root/.fastai/data/mnist_sample/train/3/10042.png'),Path('/root/.fastai/data/mnist_sample/train/3/10052.png'),Path('/root/.fastai/data/mnist_sample/train/3/1007.png'),Path('/root/.fastai/data/mnist_sample/train/3/10074.png'),Path('/root/.fastai/data/mnist_sample/train/3/10091.png'),Path('/root/.fastai/data/mnist_sample/train/3/10093.png'),Path('/root/.fastai/data/mnist_sample/train/3/10097.png'),Path('/root/.fastai/data/mnist_sample/train/3/10099.png'),Path('/root/.fastai/data/mnist_sample/train/3/10116.png'),Path('/root/.fastai/data/mnist_sample/train/3/10125.png'),Path('/root/.fastai/data/mnist_sample/train/3/10137.png'),Path('/root/.fastai/data/mnist_sample/train/3/10141.png'),Path('/ro

In [46]:
from PIL import Image
three_rand = np.random.randint(0,len(threes),4)

In [48]:
Image.open(threes[three_rand[0]])

In [49]:
Image.open(threes[three_rand[1]])

In [50]:
Image.open(threes[three_rand[2]])

In [55]:
Image.open(threes[three_rand[3]])

In [57]:

m7 = sevens[seven_rand[1]]

## 1.2 Converting images from .png into numbers for Numpy/Pytorch manipulation

### 1.2.1 Using one sample 3 to see the pixels

In [63]:
type(threes[three_rand[3]])

pathlib.PosixPath

In [69]:
m3 = Image.open(threes[three_rand[1]]) # type: PngImageFile object
m3t = tensor(m3) # note: m3 has to be image, not the path of the image 
m3t.shape

torch.Size([28, 28])

In [90]:
m3t_df = pd.DataFrame(m3t)
m3t_df.style.set_properties(**{'font-size':'7pt'}).background_gradient('Greys')  # can also change to pink

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,154,161,67,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,126,240,249,113,0,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,36,190,251,70,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,27,237,229,13,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,180,254,106,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,27,254,113,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,7,225,179,0,0,0,0,0,0,0,0,0


In [76]:
m3t_df.iloc[3:4,13:15]

,13,14
3,154,161


😆 Interesting: I thought that the image would contains values of 1s and 0s, with 1s being the blocks that are dark. But it turns out that there're different shades of darkness: very dark takes values 200 t0 254, dark grey are 144 to 188, and then light grey 50 to 140, and very very light grey is within 100.

### 1.2.2 Let's see if all the images under Train and Validation as the same size: 28x28

In [78]:
# TODO: find out the different shapes of images using Parallel
from fastcore.parallel import *

train_files = get_image_files(path/'train')
# print(train_files) # don't run this line, it will print out all the files in this folder

def file_size(image): return PILImage.create(image).size

train_file_size = parallel(file_size, train_files, n_workers=10)
pd.Series(train_file_size).value_counts()

(28, 28)    12396
Name: count, dtype: int64

In [79]:
valid_files = get_image_files(path/'valid')
valid_file_size = parallel(file_size,valid_files, n_workers=10)
pd.Series(valid_file_size).value_counts()

(28, 28)    2038
Name: count, dtype: int64

# Section 2: The Basics of Solving Linear Model

## 2.1 Loading the image data into tensors